In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier, StackingClassifier
import pandas as pd
import numpy as np

## Загружаем данные

In [11]:
# Load data again
file_path = 'data/1/eeg_data.csv'
data = pd.read_csv(file_path)
# data.rename(columns = {'epileptic_': 'target'})
data.head()

,FP1_power_delta,FP1_power_theta,FP1_power_alpha,FP1_power_beta,FP1_power_gamma,FP2_power_delta,FP2_power_theta,FP2_power_alpha,FP2_power_beta,FP2_power_gamma,...,T5_STD_D4,T5_STD_D5,T5_STD_D6,T6_STD_D1,T6_STD_D2,T6_STD_D3,T6_STD_D4,T6_STD_D5,T6_STD_D6,epileptic_
0,15.980332,8.633358,3.092433,0.846559,0.372832,17.581626,9.937238,3.485713,0.945396,0.428797,...,31.427918,29.699413,32.013546,3.583559,12.529528,45.831132,61.890813,34.783454,33.245906,1
1,223.267803,41.573241,13.043525,6.463381,6.168712,298.274432,50.013025,14.411482,6.659230,4.942250,...,24.822720,47.185918,93.619195,14.158489,15.217886,30.927407,77.324322,153.442359,213.780950,1
2,158.673907,31.788608,9.956198,3.736977,2.926979,232.762741,39.332981,10.956081,3.478036,2.153082,...,21.708222,47.412375,85.729645,9.493639,11.824569,24.555772,59.229187,112.406306,171.618208,1
3,29.767326,3.875543,1.553127,0.438266,0.140495,45.206547,5.173542,2.028637,0.528962,0.161283,...,26.319506,39.628894,51.473202,1.872515,5.392639,17.296780,35.824586,44.892899,79.613460,1
4,33.821075,3.635210,1.347280,0.313205,0.080396,55.670043,4.925017,1.783729,0.394463,0.102254,...,26.111048,39.177537,52.925676,1.408291,4.556997,12.733549,25.676801,44.406617,94.770581,1


In [12]:
# Define your target column here (assuming the last column represents sleep phase labels)
# Replace 'Target' with the actual target column name for sleep phase classification
target_column = 'epileptic_'  # Placeholder name; replace with the actual column name

# Separate features and target
X = data.drop(columns=[target_column])
y = data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Models

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, SimpleRNN, Dropout
from sklearn.metrics import accuracy_score

In [14]:
# Определение функции потерь Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt + tf.keras.backend.epsilon())
    return focal_loss_fixed

In [25]:
model1 = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train_scaled.shape[1], 1)),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train_scaled[..., np.newaxis], y_train, epochs=1, batch_size=32, verbose=1)

model2 = Sequential()
model2.add(SimpleRNN(64, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model2.add(SimpleRNN(32, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
# model2.add(Dropout(0.6))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train_scaled[..., np.newaxis], y_train, epochs=1, batch_size=32, verbose=1)

model3 = Sequential()
model3.add(Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model3.add(Dense(16, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model3.fit(X_train_scaled, y_train, epochs=1, batch_size=32, verbose=1)

 4/56 [=>............................] - ETA: 3:11 - loss: 0.6906 - accuracy: 0.6328

KeyboardInterrupt: 

In [27]:
pred_train1 = model1.predict(X_train_scaled[..., np.newaxis])
pred_train2 = model2.predict(X_train_scaled[..., np.newaxis])
pred_train3 = model3.predict(X_train_scaled)

pred_test1 = model1.predict(X_test_scaled[..., np.newaxis])
pred_test2 = model2.predict(X_test_scaled[..., np.newaxis])
pred_test3 = model3.predict(X_test_scaled)

# Объединение предсказаний для обучения логистической регрессии
stacked_predictions_train = np.hstack([pred_train1, pred_train2, pred_train3])
stacked_predictions_test = np.hstack([pred_test1, pred_test2, pred_test3])
stacked_predictions_train

14/14 [==============================] - 0s 8ms/step


array([[0.5045496 , 0.5408978 , 0.5187068 ],
       [0.508897  , 0.5554257 , 0.57199776],
       [0.27748528, 0.41797104, 0.40897095],
       ...,
       [0.49839798, 0.5991533 , 0.4078643 ],
       [0.28624314, 0.23571089, 0.38419157],
       [0.41224277, 0.49348524, 0.46176916]], dtype=float32)

In [28]:
# Линейная регрессия в качестве финальной модели
final_model = LogisticRegression()
final_model.fit(stacked_predictions_train, y_train)

LogisticRegression()

In [31]:
# Оценка модели
final_predictions = final_model.predict(stacked_predictions_test)
accuracy = accuracy_score(y_test, final_predictions)
print(f'Final Ensemble Model Accuracy: {accuracy:.4f}')

Final Ensemble Model Accuracy: 0.6509
